(assignment04)=

# Assignment #4 (demo). Exploring OLS, Lasso and Random Forest in a regression task

<img src="https://habrastorage.org/webt/ia/m9/zk/iam9zkyzqebnf_okxipihkgjwnw.jpeg" />

Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

**Same assignment as a [Kaggle Notebook](https://www.kaggle.com/kashnitsky/a6-demo-linear-models-and-rf-for-regression) + [solution](https://www.kaggle.com/kashnitsky/a6-demo-regression-solution).**  

```{figure} /_static/img/wine_quality.jpg
:width: 444px
```

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [4]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
DATA_PATH = "../../data/"

In [5]:
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [6]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [9]:
y = data['quality']
X = data.drop('quality', axis=1)

In [11]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [14]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [18]:
train_pred = mean_squared_error(y_train, linreg.predict(X_train_scaled))
valid_pred = mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled))
print(f"Mean squared error (train):{train_pred: .3f}")
print(f"Mean squared error (test): {valid_pred: .3f}")

Mean squared error (train): 0.558
Mean squared error (test):  0.584


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [21]:
linreg.coef_

(array([ 9.78219223e-02, -1.92259947e-01, -1.83224449e-04,  5.38164096e-01,
         8.12724353e-03,  4.21804406e-02,  1.43040227e-02, -6.65720472e-01,
         1.50036006e-01,  6.20533605e-02,  1.29533447e-01]),
 {'copy_X': True,
  'fit_intercept': True,
  'n_jobs': None,
  'normalize': 'deprecated',
  'positive': False})

In [29]:
linreg_coef = pd.DataFrame(linreg.coef_, index=X_train.columns)
linreg_coef
linreg_coef.sort_values(by=0, key=lambda x: np.abs(x), ascending=False)

,0
density,-0.665720
residual sugar,0.538164
volatile acidity,-0.192260
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127


## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [31]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [33]:
lasso1_coef = pd.DataFrame(lasso1.coef_, index=X_train.columns)
lasso1_coef.sort_values(by=0, key=lambda x: np.abs(x), ascending=False)

,0
alcohol,0.322425
residual sugar,0.256363
density,-0.235492
volatile acidity,-0.188479
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
chlorides,-0.002747
fixed acidity,-0.000000
citric acid,-0.000000


**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [34]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17, cv=5)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [35]:
lasso_cv.alpha_

0.0002833096101839324

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [36]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, index=X_train.columns)
lasso_cv_coef.sort_values(by=0, key=lambda x: np.abs(x), ascending=False)

,0
density,-0.648161
residual sugar,0.526883
volatile acidity,-0.192049
pH,0.146549
alcohol,0.137115
fixed acidity,0.093295
sulphates,0.060939
free sulfur dioxide,0.042698
total sulfur dioxide,0.012969
chlorides,0.006933


**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [37]:
train_pred = mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))
valid_pred = mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))
print(f"Mean squared error (train):{train_pred: .3f}")
print(f"Mean squared error (test): {valid_pred: .3f}")

Mean squared error (train): 0.558
Mean squared error (test):  0.583


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [38]:
forest = RandomForestRegressor(random_state=17)
forest_scaled = RandomForestRegressor(random_state=17)
forest.fit(X_train, y_train)
forest_scaled.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [41]:
train_scaled_forest_err = cross_val_score(forest_scaled, X_train_scaled, y_train, scoring='neg_mean_squared_error')
train_scaled_forest_err.mean()

-0.4142003732204039

In [43]:
mse_scaled_forest = mean_squared_error(y_train, forest_scaled.predict(X_train_scaled))
train_scaled_forest_err = cross_val_score(forest_scaled, X_train_scaled, y_train, scoring='neg_mean_squared_error')
test_scaled_forest_err = mean_squared_error(y_holdout, forest_scaled.predict(X_holdout_scaled))

print("Mean squared error (train): %.3f" % mse_scaled_forest)
print("Mean squared error (cv): %.3f" % train_scaled_forest_err.mean())
print("Mean squared error (test): %.3f" % test_scaled_forest_err)

print('\nnot scaled:')

mse_forest = mean_squared_error(y_train, forest.predict(X_train))
train_forest_err = cross_val_score(forest, X_train, y_train, scoring='neg_mean_squared_error')
test_forest_err = mean_squared_error(y_holdout, forest.predict(X_holdout))

print("Mean squared error (train): %.3f" % mse_forest)
print("Mean squared error (cv): %.3f" % train_forest_err.mean())
print("Mean squared error (test): %.3f" % test_forest_err)



Mean squared error (train): 0.053
Mean squared error (cv): -0.414
Mean squared error (test): 0.372

not scaled:
Mean squared error (train): 0.053
Mean squared error (cv): -0.414
Mean squared error (test): 0.371


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [ ]:
%%time
forest_params = {'max_depth': list(range(10, 25)),
#                  'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(random_state=17, verbose=1), param_grid=forest_params, n_jobs=-1, verbose=1, scoring='neg_mean_squared_error')
locally_best_forest.fit(X_train_scaled, y_train)

In [49]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6}, -0.39773288191505934)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [51]:
train_forest_err = cross_val_score(locally_best_forest.best_estimator_, X_train_scaled, y_train, scoring='neg_mean_squared_error')
test_forest_err = mean_squared_error(y_holdout, locally_best_forest.predict(X_holdout_scaled))

print("Mean squared error (cv): %.3f" % np.abs(train_forest_err).mean())
print("Mean squared error (test): %.3f" % test_forest_err)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

Mean squared error (cv): 0.398
Mean squared error (test): 0.366


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [55]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, index=X_train.columns)
rf_importance.sort_values(by=0, key= lambda x: np.abs(x), ascending=False) 

,0
alcohol,0.206056
volatile acidity,0.117578
free sulfur dioxide,0.111556
density,0.088549
pH,0.073659
total sulfur dioxide,0.073640
chlorides,0.073366
residual sugar,0.072072
citric acid,0.062601
fixed acidity,0.061813


**Make conclusions about the performance of the explored 3 models in this particular prediction task.**